In [1]:
import pandas as pd
from sklearn import ensemble
import seaborn as sns
from matplotlib import pyplot as plt
import numpy as np
from sklearn import preprocessing

columnName = ['seed', 'lat', 'long'	, 'fg2pct', 'fg3pct', 'ftpct', 'blockpct', 'oppfg2pct', 'oppfg3pct', 'oppftpct',
              'oppblockpct', 'f3grate', 'oppf3grate', 'arate', 'opparate', 'stlrate', 'oppstlrate', 'tempo', 'adjtempo',
              'oe', 'adjoe', 'de', 'adjde', 'rpi_rating', 'pt_school_ncaa', 'pt_overall_ncaa', 'pt_school_s16', 'pt_overall_s16',
              'pt_school_ff', 'pt_overall_ff', 'pt_career_school_wins',	'pt_career_school_losses', 'pt_career_overall_losses',
              'team_season_wins', 'pt_team_season_losses', 'pt_coach_season_wins', 'pt_coach_season_losses']

df = pd.read_csv('NCAA_Tourney_2002-2016_1.csv')
features = df.iloc[:, :-4]
results = df.iloc[:, -4:]

df_team1 = df.iloc[:, 2:39]
df_team1.columns = columnName
df_team2 = df.iloc[:, 39:76]
df_team2.columns = columnName

df_team1.loc[:, 'expwin'] = df.loc[:, 'team1_adjoe'] ** 11.5 / (df.loc[:, 'team1_adjde'] ** 11.5
                                                            + df.loc[:, 'team1_adjoe'] ** 11.5)


df_team2.loc[:, 'expwin'] = df.loc[:, 'team2_adjoe'] ** 11.5 / (df.loc[:, 'team2_adjde'] ** 11.5
                                                            + df.loc[:, 'team2_adjoe'] ** 11.5)


df_team1.loc[:, 'log5'] = (df_team1.loc[:, 'expwin'] - df_team1.loc[:, 'expwin'] * df_team2.loc[:, 'expwin']) / \
            ((df_team1.loc[:, 'expwin'] + df_team2.loc[:, 'expwin'] - 2* df_team1.loc[:, 'expwin'] * df_team2.loc[:, 'expwin']))

df_team2.loc[:, 'log5'] = (df_team2.loc[:, 'expwin'] - df_team1.loc[:, 'expwin'] * df_team2.loc[:, 'expwin']) / \
            ((df_team1.loc[:, 'expwin'] + df_team2.loc[:, 'expwin'] - 2* df_team1.loc[:, 'expwin'] * df_team2.loc[:, 'expwin']))


data = df_team1 - df_team2

#normolizer = preprocessing.Normalizer().fit(data)
#df_norm = pd.DataFrame(normolizer.transform(data))
#df_norm.columns = ['seed', 'lat', 'long'	'fg2pct', 'fg3pct', 'ftpct', 'blockpct', 'oppfg2pct', 'oppfg3pct', 'oppftpct',
#              'oppblockpct', 'f3grate', 'oppf3grate', 'arate', 'opparate', 'stlrate', 'oppstlrate', 'tempo', 'adjtempo',
#              'oe', 'adjoe', 'de', 'adjde', 'rpi_rating', 'pt_school_ncaa', 'pt_overall_ncaa', 'pt_school_s16', 'pt_overall_s16',
#              'pt_school_ff', 'pt_overall_ff', 'pt_career_school_wins',	'pt_career_school_losses', 'pt_career_overall_losses',
#              'team_season_wins', 'pt_team_season_losses', 'pt_coach_season_wins', 'pt_coach_season_losses', 'expwin','log5']

X = data
y = results.loc[:, 'result']
X_train = pd.DataFrame()
X_test = pd.DataFrame()
y_train = pd.DataFrame()
y_test = pd.DataFrame()
X_valid = pd.DataFrame()
y_valid = pd.DataFrame()


for year in range(2002, 2014):
    X_train = pd.concat([X_train, X.loc[results.Season == year]])
    y_train = pd.concat([y_train, y.loc[results.Season == year]])

for year in range(2014, 2017):
    X_test = pd.concat([X_test, X.loc[results.Season == year]])
    y_test = pd.concat([y_test, y.loc[results.Season == year]])
    
paramsGB = {'n_estimators': 10, 'max_depth': 5, 'min_samples_split': 2, 'loss': 'deviance'}

original_params = {'n_estimators': 1000, 'max_leaf_nodes': 4, 'max_depth': 5, 'random_state': 2,
                   'min_samples_split': 5}

clf = ensemble.GradientBoostingClassifier(**original_params)

clf.fit(X_train, y_train.values.ravel())

y_pred = clf.predict(X_test)

sns.set(font_scale=1, rc={"lines.linewidth": 1.2})

fig = plt.figure(figsize=(20, 10), dpi=80)
fig.suptitle('Predictor Importance', fontsize=20)

plt.subplot(1,1,1)

f = clf.feature_importances_
%matplotlib
f = 100.0 * (f / f.max())
indices = np.argsort(f)[-30:]
plt.barh(np.arange(30), f[indices],color='dodgerblue',alpha=.4)
plt.yticks(np.arange(30 + 0.25), np.array(X.columns)[indices])
plt.xlabel('Relative importance'), \
plt.title('Top Ten Important Variables')

Using matplotlib backend: MacOSX


(<matplotlib.text.Text at 0x11d78ea20>, <matplotlib.text.Text at 0x11f268f60>)

In [2]:
from sklearn import linear_model, preprocessing
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import log_loss
from sklearn.pipeline import make_pipeline
from sklearn.naive_bayes import GaussianNB
from sklearn.decomposition import PCA
from sklearn.neural_network import BernoulliRBM
from sklearn.gaussian_process.kernels import RBF
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split

for i in range(3,35):
    columns = list(X.columns[np.argsort(f)[-i:]])
    newFeatures = data.loc[:, columns]
    X_new = newFeatures
    y_new = results.loc[:, 'result']
    X_train = pd.DataFrame()
    X_test = pd.DataFrame()
    y_train = pd.DataFrame()
    y_test = pd.DataFrame()
    X_valid = pd.DataFrame()
    y_valid = pd.DataFrame()
    
    
    #normolizer = preprocessing.Normalizer().fit(X_new)
    #X_new = pd.DataFrame(normolizer.transform(X_new))
 
    for year in range(2002, 2015):
        X_train = pd.concat([X_train, X_new.loc[results.Season == year]])
        y_train = pd.concat([y_train, y_new.loc[results.Season == year]])
        
    for year in range(2002, 2015):
        X_valid = pd.concat([X_valid, X_new.loc[results.Season == year]])
        y_valid = pd.concat([y_valid, y_new.loc[results.Season == year]])
    
    for year in range(2015, 2017):
        X_test = pd.concat([X_test, X_new.loc[results.Season == year]])
        y_test = pd.concat([y_test, y_new.loc[results.Season == year]])
        
    
    clf = linear_model.LogisticRegressionCV(Cs=2, solver='liblinear', max_iter=500, tol=1e-6, scoring='neg_log_loss')
    #clf.fit(X_train, y_train.values.ravel())
    #clf = linear_model.LogisticRegression()

    for p in range(3,i):
        pca = PCA(p)
    
        pipeline = make_pipeline(pca, clf)
        pipeline.fit(X_train, y_train.values.ravel())
        
        sig_clf = CalibratedClassifierCV(pipeline, method="sigmoid", cv='prefit')
        sig_clf.fit(X_valid, y_valid.values.ravel())
        
        y_pred = sig_clf.predict_proba(X_test)
        logLoss = log_loss(y_true=y_test, y_pred=y_pred)
        
        print(logLoss, '\t\t', i, '\t\t', p)

0.628340361213 		 4 		 3
0.627175859654 		 5 		 3
0.631392848057 		 5 		 4
0.629898939197 		 6 		 3
0.621171058376 		 6 		 4


0.63296211785 		 6 		 5
0.614561739841 		 7 		 3
0.623949858075 		 7 		 4
0.614073923198 		 7 		 5
0.625618949649 		 7 		 6


0.633889134302 		 8 		 3
0.622786592284 		 8 		 4
0.628275072557 		 8 		 5
0.617066179778 		 8 		 6
0.628263468553 		 8 		 7


0.641147866907 		 9 		 3
0.6312015122 		 9 		 4
0.622023691618 		 9 		 5
0.623850467784 		 9 		 6
0.610752079283 		 9 		 7
0.613387695238 		 9 		 8


0.682390935606 		 10 		 3
0.660022335488 		 10 		 4
0.625108798632 		 10 		 5
0.627189938655 		 10 		 6
0.627120143548 		 10 		 7
0.613929408915 		 10 		 8
0.619163716498 		 10 		 9


...

0.582233000487 		 34 		 25
0.589359610362 		 34 		 26
0.589319621081 		 34 		 27
0.58934713061 		 34 		 28


0.596218014265 		 34 		 29


0.59828204718 		 34 		 30
0.539057717686 		 34 		 31


0.539057727898 		 34 		 32


0.539057731469 		 34 		 33


In [ ]:
columnName = ['seed', 'lat', 'long'	'fg2pct', 'fg3pct', 'ftpct', 'blockpct', 'oppfg2pct', 'oppfg3pct', 'oppftpct',
              'oppblockpct', 'f3grate', 'oppf3grate', 'arate', 'opparate', 'stlrate', 'oppstlrate', 'tempo', 'adjtempo',
              'oe', 'adjoe', 'de', 'adjde', 'rpi_rating', 'pt_school_ncaa', 'pt_overall_ncaa', 'pt_school_s16', 'pt_overall_s16',
              'pt_school_ff', 'pt_overall_ff', 'pt_career_school_wins',	'pt_career_school_losses', 'pt_career_overall_losses',
              'team_season_wins', 'pt_team_season_losses', 'pt_coach_season_wins', 'pt_coach_season_losses']

df = pd.read_csv('NCAA_Tourney_2002-2016_1.csv')
features = df.iloc[:, :-4]
results = df.iloc[:, -4:]

df_team1 = df.iloc[:, 2:38]
df_team1.columns = columnName
df_team2 = df.iloc[:, 39:75]
df_team2.columns = columnName

df_team1.loc[:, 'expwin'] = df.loc[:, 'team1_adjoe'] ** 11.5 / (df.loc[:, 'team1_adjde'] ** 11.5
                                                            + df.loc[:, 'team1_adjoe'] ** 11.5)


df_team2.loc[:, 'expwin'] = df.loc[:, 'team2_adjoe'] ** 11.5 / (df.loc[:, 'team2_adjde'] ** 11.5
                                                            + df.loc[:, 'team2_adjoe'] ** 11.5)


df_team1.loc[:, 'log5'] = (df_team1.loc[:, 'expwin'] - df_team1.loc[:, 'expwin'] * df_team2.loc[:, 'expwin']) / \
            ((df_team1.loc[:, 'expwin'] + df_team2.loc[:, 'expwin'] - 2* df_team1.loc[:, 'expwin'] * df_team2.loc[:, 'expwin']))

df_team2.loc[:, 'log5'] = (df_team2.loc[:, 'expwin'] - df_team1.loc[:, 'expwin'] * df_team2.loc[:, 'expwin']) / \
            ((df_team1.loc[:, 'expwin'] + df_team2.loc[:, 'expwin'] - 2* df_team1.loc[:, 'expwin'] * df_team2.loc[:, 'expwin']))


data = df_team1 - df_team2

In [24]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

estimators = [('reduce_dim', PCA()), ('clf', SVC())]
pipe = Pipeline(estimators)


params = dict(reduce_dim=[None, PCA(10), PCA(15)],
              clf=[SVC(), LogisticRegression(), linear_model.LogisticRegressionCV()])
grid_search = GridSearchCV(pipe, param_grid=params)

In [26]:
grid_search.fit(X_train, y_train.values.ravel())
y_pred = grid_search.predict_proba(X_test)

In [27]:
log_loss(y_test,y_pred)

0.59712569568627594

In [ ]:
n_factor = 30
n_pc = 10